In [1]:
var('t q')

def number_of_ones(partition):
    for i in range(len(partition)):
        if partition[len(partition)-i-1] != 1:
            return i
    return len(partition)

def my_len(partition):
    return 2*len(partition)-number_of_ones(partition)

def series_t_q(list_of_list_of_partition):
    series = 0
    for i in range(len(list_of_list_of_partition)):
        series_t = 0
        for j in range(len(list_of_list_of_partition[i])):
            series_t = series_t+t^my_len(list_of_list_of_partition[i][j])
        series = series+series_t*q^i
    return series

def partition_ends(partition, tail, is_greater):
    if len(partition) < len(tail)-1:
        return False
    partition2 = partition[len(partition)-len(tail)+1: len(partition)]
    if partition2 != tail[1:]:
        return False
    if is_greater:
        if partition == partition2:
            return True
        if partition[len(partition)-len(tail)] > tail[0]:
            return True
    else:
        if partition == partition2:
            return False
        if partition[len(partition)-len(tail)] == tail[0]:
            return True
    return False

def list_of_list_of_partition_ends(list_of_list_of_partition, tail, is_greater):
    return [[partition
             for partition in list_of_partition if partition_ends(partition, tail, is_greater)]
            for list_of_partition in list_of_list_of_partition]

def vector_sum(vector1, vector2):
    return {partition: vector1.get(partition, 0)+vector2.get(partition, 0)
            for partition in set(vector1)|set(vector2)}

def vector_simplify0(partition):
    for i in range(len(partition)-1):
        if partition[i] < partition[i+1]:
            return {partition[0: i]+(partition[i]+partition[i+1],)+partition[i+2: len(partition)]:
                    partition[i+1]-partition[i], partition[0: i]+(partition[i+1],)+(partition[i],)
                    +partition[i+2: len(partition)]: 1}
    return {partition: 1}

def vector_simplify(vector):
    vector2 = {}
    for partition in set(vector):
        vector3 = vector_simplify0(partition)
        vector2 = vector_sum(vector2, {partition2:vector3[partition2]*vector[partition]
                                       for partition2 in set(vector3)})
    if vector2 == vector:
        return vector2
    return vector_simplify(vector2)

def L_partition_vector(partition, vector):
    return vector_simplify(
        {partition+partition2: vector[partition2] for partition2 in set(vector)})

def my_order(a):
    polynomial = to_polynomial(a)
    polynomial = [-k for k in polynomial]
    polynomial.reverse()
    return ((my_len(a), len(a)-number_of_ones(a), polynomial)) 

def ordered_partitions(n):
    if n <= 0:
        return [[]]
    list_of_partition = list(Partitions(n))
    list_of_partition.sort(key = my_order, reverse = True)
    return list_of_partition

def is_subpartition(partition, partition2):
    if len(partition) < len(partition2):
        return False
    for i in range(len(partition)-len(partition2)+1):
        if partition[i: i+len(partition2)] == partition2:
            return True
    return False

def partition_contains_any_of_list(partition, list_of_partition):
    for partition2 in list_of_partition:
        if is_subpartition(partition, partition2):
            return True
    return False

def partition_contains_any_of_Q(partition):
    if partition == []:
        return False
    if partition_contains_any_of_list(
        partition,
        [[2], [1, 1, 1, 1], [3, 1, 1, 1], [3, 3, 1], [4, 3, 1], [4, 4, 1, 1], [5, 4, 1, 1, 1], [5, 5, 1, 1, 1], [6, 5, 3, 1, 1], [6, 6, 3, 1, 1], [7, 6, 4, 1, 1, 1], [8, 7, 5, 3, 1, 1]]):
        return True
    if len(partition) > 2:
        for i in range(len(partition)-2):
            if (partition[i]-partition[i+2] <= 2 and partition[i+2] > 1):
                return True
    if len(partition) > 3:
        for i in range(len(partition)-3):
            if ((partition[i]-partition[i+3] <= 3 and partition[i+3] > 1)
                or (partition[i]-partition[i+3] <= 4 and partition[i+1]-partition[i+2] <= 3 and partition[i+3] > 1)):
                return True
    if len(partition) > 4:
        for i in range(len(partition)-4):
            if (partition[i]-partition[i+1] == 1 and partition[i+1]-partition[i+2] == 2
                and partition[i+2]-partition[i+3] == 2 and partition[i+3]-partition[i+4] == 1 and partition[i+4] > 1):
                return True
    return False

def matrix_UVir_m2_m3(n):
    m2 = {(1, 1): -4, (2,): 3}
    m4 = {(1, 1, 1, 1):  144, (2, 1, 1): -600, (2, 2): 49, (3, 1): 264, (4,): -36}
    list_of_partition = ordered_partitions(n)
    list_of_partition2 = ordered_partitions(n-2)
    list_of_partition4 = ordered_partitions(n-4)
    list_of_vector_m2 = [L_partition_vector(tuple(list_of_partition2[i]), m2)
                         for i in range(len(list_of_partition2))]
    list_of_vector_m4 = [L_partition_vector(tuple(list_of_partition4[i]), m4)
                         for i in range(len(list_of_partition4))]
    matrix_m2 = matrix({(i, j): list_of_vector_m2[i].get(tuple(list_of_partition[j]), 0)
                        for j in range(len(list_of_partition))
                        for i in range(len(list_of_partition2))}, sparse = False)
    matrix_m4 = matrix({(i, j): list_of_vector_m4[i].get(tuple(list_of_partition[j]), 0)
                        for j in range(len(list_of_partition))
                        for i in range(len(list_of_partition4))}, sparse = False)
    matrix_m2 = matrix_m2.stack(matrix_m4)
    matrix_m2 = matrix_m2.rref()
    return matrix_m2

def to_polynomial(partition):
    polynomial = [0]*(sum(partition)-1)
    for i in range(len(partition)-number_of_ones(partition)):
        polynomial[partition[i]-2] += 1
    return polynomial

def pivots_up_to(n):
    list_of_list_of_partition = []
    for i in range(n + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [list_of_partition[j] for j in matrix_UVir_m2_m3(i).pivots()])
    return list_of_list_of_partition

def basis_up_to(n):
    list_of_list_of_partition = []
    pivots = pivots_up_to(n)
    for i in range(n + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [partition for partition in list_of_partition if partition not in pivots[i]])
    return list_of_list_of_partition

def basis_up_tov2(n):
    list_of_list_of_partition = []
    for i in range(n + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [partition for partition in list_of_partition
             if not partition_contains_any_of_Q(partition)])
    return list_of_list_of_partition

In [2]:
ordered_partitions(15)

[[2, 2, 2, 2, 2, 2, 2, 1],
 [2, 2, 2, 2, 2, 2, 1, 1, 1],
 [2, 2, 2, 2, 2, 1, 1, 1, 1, 1],
 [2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1],
 [2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [3, 2, 2, 2, 2, 2, 2],
 [3, 2, 2, 2, 2, 2, 1, 1],
 [3, 2, 2, 2, 2, 1, 1, 1, 1],
 [3, 2, 2, 2, 1, 1, 1, 1, 1, 1],
 [3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1],
 [3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [3, 3, 2, 2, 2, 2, 1],
 [4, 2, 2, 2, 2, 2, 1],
 [3, 3, 2, 2, 2, 1, 1, 1],
 [4, 2, 2, 2, 2, 1, 1, 1],
 [3, 3, 2, 2, 1, 1, 1, 1, 1],
 [4, 2, 2, 2, 1, 1, 1, 1, 1],
 [3, 3, 2, 1, 1, 1, 1, 1, 1, 1],
 [4, 2, 2, 1, 1, 1, 1, 1, 1, 1],
 [3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [3, 3, 3, 2, 2, 2],
 [4, 3, 2, 2, 2, 2],
 [5, 2, 2, 2, 2, 2],
 [3, 3, 3, 2, 2, 1, 1],
 [4, 3, 2, 2, 2, 1, 1],
 [5, 2, 2, 2, 2, 1,

In [3]:
pivots_up_to(25)

[[],
 [],
 [[2]],
 [[2, 1]],
 [[2, 2], [2, 1, 1], [1, 1, 1, 1]],
 [[2, 2, 1], [2, 1, 1, 1], [1, 1, 1, 1, 1], [3, 2]],
 [[2, 2, 2],
  [2, 2, 1, 1],
  [2, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1],
  [3, 2, 1],
  [3, 1, 1, 1],
  [4, 2]],
 [[2, 2, 2, 1],
  [2, 2, 1, 1, 1],
  [2, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1],
  [3, 2, 2],
  [3, 2, 1, 1],
  [3, 1, 1, 1, 1],
  [3, 3, 1],
  [4, 2, 1],
  [5, 2]],
 [[2, 2, 2, 2],
  [2, 2, 2, 1, 1],
  [2, 2, 1, 1, 1, 1],
  [2, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1],
  [3, 2, 2, 1],
  [3, 2, 1, 1, 1],
  [3, 1, 1, 1, 1, 1],
  [3, 3, 2],
  [4, 2, 2],
  [3, 3, 1, 1],
  [4, 2, 1, 1],
  [4, 1, 1, 1, 1],
  [4, 3, 1],
  [5, 2, 1],
  [6, 2]],
 [[2, 2, 2, 2, 1],
  [2, 2, 2, 1, 1, 1],
  [2, 2, 1, 1, 1, 1, 1],
  [2, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1],
  [3, 2, 2, 2],
  [3, 2, 2, 1, 1],
  [3, 2, 1, 1, 1, 1],
  [3, 1, 1, 1, 1, 1, 1],
  [3, 3, 2, 1],
  [4, 2, 2, 1],
  [3, 3, 1, 1, 1],
  [4, 2, 1, 1, 1],
  [4, 1, 1, 1, 1, 1],
  [3, 3, 3],
  [4, 3, 2],

In [4]:
%time w = basis_up_to(27)

CPU times: user 17min 41s, sys: 11.9 s, total: 17min 53s
Wall time: 17min 53s


In [5]:
w

[[[]],
 [[1]],
 [[1, 1]],
 [[1, 1, 1], [3]],
 [[3, 1], [4]],
 [[3, 1, 1], [4, 1], [5]],
 [[3, 3], [4, 1, 1], [5, 1], [6]],
 [[4, 1, 1, 1], [4, 3], [5, 1, 1], [6, 1], [7]],
 [[5, 1, 1, 1], [4, 4], [5, 3], [6, 1, 1], [7, 1], [8]],
 [[4, 4, 1], [5, 3, 1], [6, 1, 1, 1], [5, 4], [6, 3], [7, 1, 1], [8, 1], [9]],
 [[5, 3, 1, 1],
  [5, 4, 1],
  [6, 3, 1],
  [7, 1, 1, 1],
  [5, 5],
  [6, 4],
  [7, 3],
  [8, 1, 1],
  [9, 1],
  [10]],
 [[5, 4, 1, 1],
  [6, 3, 1, 1],
  [5, 5, 1],
  [6, 4, 1],
  [7, 3, 1],
  [8, 1, 1, 1],
  [6, 5],
  [7, 4],
  [8, 3],
  [9, 1, 1],
  [10, 1],
  [11]],
 [[6, 3, 3],
  [5, 5, 1, 1],
  [6, 4, 1, 1],
  [7, 3, 1, 1],
  [6, 5, 1],
  [7, 4, 1],
  [8, 3, 1],
  [9, 1, 1, 1],
  [6, 6],
  [7, 5],
  [8, 4],
  [9, 3],
  [10, 1, 1],
  [11, 1],
  [12]],
 [[6, 4, 1, 1, 1],
  [6, 4, 3],
  [7, 3, 3],
  [6, 5, 1, 1],
  [7, 4, 1, 1],
  [8, 3, 1, 1],
  [6, 6, 1],
  [7, 5, 1],
  [8, 4, 1],
  [9, 3, 1],
  [10, 1, 1, 1],
  [7, 6],
  [8, 5],
  [9, 4],
  [10, 3],
  [11, 1, 1],
  [12, 1],
  [1

In [6]:
w == basis_up_tov2(27)

True

In [7]:
%time series_t_q(basis_up_tov2(50))

CPU times: user 58.8 s, sys: 1.38 s, total: 1min
Wall time: 1min


(7*t^14 + 126*t^13 + 434*t^12 + 612*t^11 + 936*t^10 + 523*t^9 + 620*t^8 + 170*t^7 + 181*t^6 + 23*t^5 + 24*t^4 + t^3 + t^2)*q^50 + (2*t^14 + 95*t^13 + 351*t^12 + 536*t^11 + 834*t^10 + 487*t^9 + 575*t^8 + 163*t^7 + 173*t^6 + 23*t^5 + 23*t^4 + t^3 + t^2)*q^49 + (t^14 + 65*t^13 + 284*t^12 + 465*t^11 + 743*t^10 + 447*t^9 + 537*t^8 + 155*t^7 + 166*t^6 + 22*t^5 + 23*t^4 + t^3 + t^2)*q^48 + (46*t^13 + 223*t^12 + 402*t^11 + 657*t^10 + 414*t^9 + 496*t^8 + 148*t^7 + 158*t^6 + 22*t^5 + 22*t^4 + t^3 + t^2)*q^47 + (29*t^13 + 175*t^12 + 344*t^11 + 580*t^10 + 378*t^9 + 461*t^8 + 141*t^7 + 151*t^6 + 21*t^5 + 22*t^4 + t^3 + t^2)*q^46 + (19*t^13 + 133*t^12 + 293*t^11 + 508*t^10 + 349*t^9 + 424*t^8 + 134*t^7 + 144*t^6 + 21*t^5 + 21*t^4 + t^3 + t^2)*q^45 + (10*t^13 + 100*t^12 + 246*t^11 + 444*t^10 + 316*t^9 + 393*t^8 + 127*t^7 + 137*t^6 + 20*t^5 + 21*t^4 + t^3 + t^2)*q^44 + (6*t^13 + 72*t^12 + 206*t^11 + 384*t^10 + 290*t^9 + 359*t^8 + 121*t^7 + 130*t^6 + 20*t^5 + 20*t^4 + t^3 + t^2)*q^43 + (2*t^13 + 52*t^1

In [8]:
series_t_q(w)

(4*t^10 + 27*t^9 + 52*t^8 + 38*t^7 + 45*t^6 + 12*t^5 + 12*t^4 + t^3 + t^2)*q^27 + (2*t^10 + 19*t^9 + 44*t^8 + 34*t^7 + 41*t^6 + 11*t^5 + 12*t^4 + t^3 + t^2)*q^26 + (15*t^9 + 35*t^8 + 31*t^7 + 37*t^6 + 11*t^5 + 11*t^4 + t^3 + t^2)*q^25 + (9*t^9 + 29*t^8 + 27*t^7 + 34*t^6 + 10*t^5 + 11*t^4 + t^3 + t^2)*q^24 + (6*t^9 + 22*t^8 + 24*t^7 + 30*t^6 + 10*t^5 + 10*t^4 + t^3 + t^2)*q^23 + (3*t^9 + 17*t^8 + 21*t^7 + 27*t^6 + 9*t^5 + 10*t^4 + t^3 + t^2)*q^22 + (2*t^9 + 12*t^8 + 18*t^7 + 24*t^6 + 9*t^5 + 9*t^4 + t^3 + t^2)*q^21 + (9*t^8 + 15*t^7 + 21*t^6 + 8*t^5 + 9*t^4 + t^3 + t^2)*q^20 + (5*t^8 + 13*t^7 + 18*t^6 + 8*t^5 + 8*t^4 + t^3 + t^2)*q^19 + (3*t^8 + 10*t^7 + 16*t^6 + 7*t^5 + 8*t^4 + t^3 + t^2)*q^18 + (t^8 + 8*t^7 + 13*t^6 + 7*t^5 + 7*t^4 + t^3 + t^2)*q^17 + (6*t^7 + 11*t^6 + 6*t^5 + 7*t^4 + t^3 + t^2)*q^16 + (4*t^7 + 9*t^6 + 6*t^5 + 6*t^4 + t^3 + t^2)*q^15 + (2*t^7 + 7*t^6 + 5*t^5 + 6*t^4 + t^3 + t^2)*q^14 + (t^7 + 5*t^6 + 5*t^5 + 5*t^4 + t^3 + t^2)*q^13 + (4*t^6 + 4*t^5 + 5*t^4 + t^3 + t^2

In [9]:
series_t_q(list_of_list_of_partition_ends(w, [2], True))

(27*t^8 + 35*t^6 + 11*t^4 + t^2)*q^27 + (22*t^8 + 31*t^6 + 11*t^4 + t^2)*q^26 + (16*t^8 + 28*t^6 + 10*t^4 + t^2)*q^25 + (13*t^8 + 25*t^6 + 10*t^4 + t^2)*q^24 + (8*t^8 + 22*t^6 + 9*t^4 + t^2)*q^23 + (6*t^8 + 19*t^6 + 9*t^4 + t^2)*q^22 + (3*t^8 + 17*t^6 + 8*t^4 + t^2)*q^21 + (2*t^8 + 14*t^6 + 8*t^4 + t^2)*q^20 + (12*t^6 + 7*t^4 + t^2)*q^19 + (10*t^6 + 7*t^4 + t^2)*q^18 + (8*t^6 + 6*t^4 + t^2)*q^17 + (6*t^6 + 6*t^4 + t^2)*q^16 + (5*t^6 + 5*t^4 + t^2)*q^15 + (3*t^6 + 5*t^4 + t^2)*q^14 + (2*t^6 + 4*t^4 + t^2)*q^13 + (t^6 + 4*t^4 + t^2)*q^12 + (3*t^4 + t^2)*q^11 + (3*t^4 + t^2)*q^10 + (2*t^4 + t^2)*q^9 + (2*t^4 + t^2)*q^8 + (t^4 + t^2)*q^7 + (t^4 + t^2)*q^6 + q^5*t^2 + q^4*t^2 + q^3*t^2 + 1

In [10]:
series_t_q(list_of_list_of_partition_ends(w, [4], True))

(17*t^6 + 9*t^4 + t^2)*q^27 + (14*t^6 + 9*t^4 + t^2)*q^26 + (12*t^6 + 8*t^4 + t^2)*q^25 + (10*t^6 + 8*t^4 + t^2)*q^24 + (8*t^6 + 7*t^4 + t^2)*q^23 + (6*t^6 + 7*t^4 + t^2)*q^22 + (5*t^6 + 6*t^4 + t^2)*q^21 + (3*t^6 + 6*t^4 + t^2)*q^20 + (2*t^6 + 5*t^4 + t^2)*q^19 + (t^6 + 5*t^4 + t^2)*q^18 + (4*t^4 + t^2)*q^17 + (4*t^4 + t^2)*q^16 + (3*t^4 + t^2)*q^15 + (3*t^4 + t^2)*q^14 + (2*t^4 + t^2)*q^13 + (2*t^4 + t^2)*q^12 + (t^4 + t^2)*q^11 + (t^4 + t^2)*q^10 + q^9*t^2 + q^8*t^2 + q^7*t^2 + q^6*t^2 + q^5*t^2 + 1

In [11]:
series_t_q(list_of_list_of_partition_ends(w, [5, 4], True))

(2*t^8 + 6*t^6 + t^4)*q^27 + (t^8 + 6*t^6 + t^4)*q^26 + (5*t^6 + t^4)*q^25 + (5*t^6 + t^4)*q^24 + (4*t^6 + t^4)*q^23 + (4*t^6 + t^4)*q^22 + (3*t^6 + t^4)*q^21 + (3*t^6 + t^4)*q^20 + (2*t^6 + t^4)*q^19 + q^16*t^4 + (2*t^6 + t^4)*q^18 + q^15*t^4 + (t^6 + t^4)*q^17 + q^14*t^4 + q^13*t^4 + q^12*t^4 + q^11*t^4 + q^10*t^4 + q^4*t^2

In [12]:
series_t_q(list_of_list_of_partition_ends(w, [5, 4], False))

q^24*t^6 + q^23*t^6 + (3*t^8 + t^6)*q^27 + q^22*t^6 + (2*t^8 + t^6)*q^26 + q^21*t^6 + (t^8 + t^6)*q^25 + q^20*t^6 + q^19*t^6 + q^18*t^6 + q^17*t^6 + q^16*t^6 + q^9*t^4

In [13]:
series_t_q(list_of_list_of_partition_ends(w, [4, 4], False))

q^23*t^6 + (3*t^8 + t^6)*q^27 + q^22*t^6 + (3*t^8 + t^6)*q^26 + q^21*t^6 + (2*t^8 + t^6)*q^25 + q^20*t^6 + (2*t^8 + t^6)*q^24 + q^19*t^6 + q^18*t^6 + q^17*t^6 + q^16*t^6 + q^15*t^6 + q^8*t^4

In [49]:
series_t_q(list_of_list_of_partition_ends(w, [5, 3], True))

(5*t^8 + 7*t^6 + t^4)*q^27 + (3*t^8 + 6*t^6 + t^4)*q^26 + (2*t^8 + 6*t^6 + t^4)*q^25 + (t^8 + 5*t^6 + t^4)*q^24 + (5*t^6 + t^4)*q^23 + (4*t^6 + t^4)*q^22 + (4*t^6 + t^4)*q^21 + (3*t^6 + t^4)*q^20 + (3*t^6 + t^4)*q^19 + (2*t^6 + t^4)*q^18 + (2*t^6 + t^4)*q^17 + q^14*t^4 + (t^6 + t^4)*q^16 + q^13*t^4 + (t^6 + t^4)*q^15 + q^12*t^4 + q^11*t^4 + q^10*t^4 + q^9*t^4 + q^3*t^2

In [15]:
series_t_q(list_of_list_of_partition_ends(w, [7, 5, 3], True))

q^23*t^6 + (2*t^8 + t^6)*q^27 + q^22*t^6 + (2*t^8 + t^6)*q^26 + q^21*t^6 + (t^8 + t^6)*q^25 + q^20*t^6 + (t^8 + t^6)*q^24 + q^19*t^6 + q^18*t^6 + q^17*t^6 + q^16*t^6 + q^8*t^4

In [16]:
series_t_q(list_of_list_of_partition_ends(w, [8, 7, 5, 3], True))

q^27*t^8 + q^26*t^8 + q^25*t^8 + q^24*t^8 + q^15*t^6

In [17]:
series_t_q(list_of_list_of_partition_ends(w, [8, 7, 5, 3], False))

q^23*t^8

In [18]:
series_t_q(list_of_list_of_partition_ends(w, [6, 5, 3], False))

q^27*t^8 + q^26*t^8 + q^25*t^8 + q^24*t^8 + q^23*t^8 + q^22*t^8 + q^14*t^6

In [19]:
series_t_q(list_of_list_of_partition_ends(w, [6, 4, 3], True))

(4*t^8 + t^6)*q^27 + (3*t^8 + t^6)*q^26 + q^21*t^6 + (3*t^8 + t^6)*q^25 + q^20*t^6 + (2*t^8 + t^6)*q^24 + q^19*t^6 + (2*t^8 + t^6)*q^23 + q^18*t^6 + (t^8 + t^6)*q^22 + q^17*t^6 + q^16*t^6 + q^15*t^6 + q^14*t^6 + q^7*t^4

In [20]:
series_t_q(list_of_list_of_partition_ends(w, [6, 4, 3], False))

q^27*t^8 + q^26*t^8 + q^25*t^8 + q^24*t^8 + q^23*t^8 + q^22*t^8 + q^21*t^8 + q^13*t^6

In [21]:
series_t_q(list_of_list_of_partition_ends(w, [3, 3], False))

(5*t^8 + t^6)*q^27 + (5*t^8 + t^6)*q^26 + (4*t^8 + t^6)*q^25 + (4*t^8 + t^6)*q^24 + q^19*t^6 + (3*t^8 + t^6)*q^23 + q^18*t^6 + (3*t^8 + t^6)*q^22 + q^17*t^6 + (2*t^8 + t^6)*q^21 + q^16*t^6 + (2*t^8 + t^6)*q^20 + q^15*t^6 + q^14*t^6 + q^13*t^6 + q^12*t^6 + q^6*t^4

In [44]:
series_t_q(list_of_list_of_partition_ends(w, [6, 3, 3], False))

q^27*t^8 + q^26*t^8 + q^25*t^8 + q^24*t^8 + q^23*t^8 + q^22*t^8 + q^21*t^8 + q^20*t^8 + q^12*t^6

In [45]:
series_t_q(list_of_list_of_partition_ends(w, [6, 3, 3], True))

(4*t^8 + t^6)*q^27 + (4*t^8 + t^6)*q^26 + (3*t^8 + t^6)*q^25 + (3*t^8 + t^6)*q^24 + q^19*t^6 + (2*t^8 + t^6)*q^23 + q^18*t^6 + (2*t^8 + t^6)*q^22 + q^17*t^6 + (t^8 + t^6)*q^21 + q^16*t^6 + (t^8 + t^6)*q^20 + q^15*t^6 + q^14*t^6 + q^13*t^6 + q^6*t^4

In [22]:
series_t_q(list_of_list_of_partition_ends(w, [2, 1], True))

(13*t^9 + 29*t^7 + 11*t^5 + t^3)*q^27 + (8*t^9 + 26*t^7 + 10*t^5 + t^3)*q^26 + (6*t^9 + 23*t^7 + 10*t^5 + t^3)*q^25 + (2*t^9 + 20*t^7 + 9*t^5 + t^3)*q^24 + (t^9 + 17*t^7 + 9*t^5 + t^3)*q^23 + (15*t^7 + 8*t^5 + t^3)*q^22 + (12*t^7 + 8*t^5 + t^3)*q^21 + (10*t^7 + 7*t^5 + t^3)*q^20 + (8*t^7 + 7*t^5 + t^3)*q^19 + (6*t^7 + 6*t^5 + t^3)*q^18 + (4*t^7 + 6*t^5 + t^3)*q^17 + (3*t^7 + 5*t^5 + t^3)*q^16 + (t^7 + 5*t^5 + t^3)*q^15 + (4*t^5 + t^3)*q^14 + (4*t^5 + t^3)*q^13 + (3*t^5 + t^3)*q^12 + (3*t^5 + t^3)*q^11 + (2*t^5 + t^3)*q^10 + q^8*t^3 + (2*t^5 + t^3)*q^9 + q^7*t^3 + q^6*t^3 + q^5*t^3 + q^4*t^3 + q*t

In [23]:
series_t_q(list_of_list_of_partition_ends(w, [4, 1], True))

(14*t^7 + 9*t^5 + t^3)*q^27 + (12*t^7 + 8*t^5 + t^3)*q^26 + (10*t^7 + 8*t^5 + t^3)*q^25 + (8*t^7 + 7*t^5 + t^3)*q^24 + (6*t^7 + 7*t^5 + t^3)*q^23 + (5*t^7 + 6*t^5 + t^3)*q^22 + (3*t^7 + 6*t^5 + t^3)*q^21 + (2*t^7 + 5*t^5 + t^3)*q^20 + (t^7 + 5*t^5 + t^3)*q^19 + (4*t^5 + t^3)*q^18 + (4*t^5 + t^3)*q^17 + (3*t^5 + t^3)*q^16 + (3*t^5 + t^3)*q^15 + (2*t^5 + t^3)*q^14 + (2*t^5 + t^3)*q^13 + (t^5 + t^3)*q^12 + q^10*t^3 + (t^5 + t^3)*q^11 + q^9*t^3 + q^8*t^3 + q^7*t^3 + q^6*t^3 + q*t

In [24]:
series_t_q(list_of_list_of_partition_ends(w, [5, 4, 1], True))

(t^9 + 6*t^7 + t^5)*q^27 + (5*t^7 + t^5)*q^26 + (5*t^7 + t^5)*q^25 + (4*t^7 + t^5)*q^24 + (4*t^7 + t^5)*q^23 + (3*t^7 + t^5)*q^22 + (3*t^7 + t^5)*q^21 + q^17*t^5 + (2*t^7 + t^5)*q^20 + q^16*t^5 + (2*t^7 + t^5)*q^19 + q^15*t^5 + (t^7 + t^5)*q^18 + q^14*t^5 + q^13*t^5 + q^12*t^5 + q^11*t^5 + q^5*t^3

In [25]:
series_t_q(list_of_list_of_partition_ends(w, [5, 4, 1], False))

q^25*t^7 + q^24*t^7 + q^23*t^7 + q^22*t^7 + (2*t^9 + t^7)*q^27 + q^21*t^7 + (t^9 + t^7)*q^26 + q^20*t^7 + q^19*t^7 + q^18*t^7 + q^17*t^7 + q^10*t^5

In [26]:
series_t_q(list_of_list_of_partition_ends(w, [4, 4, 1], False))

q^24*t^7 + q^23*t^7 + q^22*t^7 + (3*t^9 + t^7)*q^27 + q^21*t^7 + (2*t^9 + t^7)*q^26 + q^20*t^7 + (2*t^9 + t^7)*q^25 + q^19*t^7 + q^18*t^7 + q^17*t^7 + q^16*t^7 + q^9*t^5

In [27]:
series_t_q(list_of_list_of_partition_ends(w, [5, 3, 1], True))

(3*t^9 + 6*t^7 + t^5)*q^27 + (2*t^9 + 6*t^7 + t^5)*q^26 + (t^9 + 5*t^7 + t^5)*q^25 + (5*t^7 + t^5)*q^24 + (4*t^7 + t^5)*q^23 + (4*t^7 + t^5)*q^22 + (3*t^7 + t^5)*q^21 + (3*t^7 + t^5)*q^20 + (2*t^7 + t^5)*q^19 + q^15*t^5 + (2*t^7 + t^5)*q^18 + q^14*t^5 + (t^7 + t^5)*q^17 + q^13*t^5 + (t^7 + t^5)*q^16 + q^12*t^5 + q^11*t^5 + q^10*t^5 + q^4*t^3

In [28]:
series_t_q(list_of_list_of_partition_ends(w, [7, 5, 3, 1], True))

q^24*t^7 + q^23*t^7 + q^22*t^7 + (2*t^9 + t^7)*q^27 + q^21*t^7 + (t^9 + t^7)*q^26 + q^20*t^7 + (t^9 + t^7)*q^25 + q^19*t^7 + q^18*t^7 + q^17*t^7 + q^9*t^5

In [29]:
series_t_q(list_of_list_of_partition_ends(w, [8, 7, 5, 3, 1], True))

q^27*t^9 + q^26*t^9 + q^25*t^9 + q^16*t^7

In [30]:
series_t_q(list_of_list_of_partition_ends(w, [8, 7, 5, 3, 1], False))

q^24*t^9

In [31]:
series_t_q(list_of_list_of_partition_ends(w, [6, 5, 3, 1], False))

q^27*t^9 + q^26*t^9 + q^25*t^9 + q^24*t^9 + q^23*t^9 + q^15*t^7

In [32]:
series_t_q(list_of_list_of_partition_ends(w, [2, 1, 1], True))

(4*t^10 + 25*t^8 + 10*t^6 + t^4)*q^27 + (2*t^10 + 22*t^8 + 10*t^6 + t^4)*q^26 + (19*t^8 + 9*t^6 + t^4)*q^25 + (16*t^8 + 9*t^6 + t^4)*q^24 + (14*t^8 + 8*t^6 + t^4)*q^23 + (11*t^8 + 8*t^6 + t^4)*q^22 + (9*t^8 + 7*t^6 + t^4)*q^21 + (7*t^8 + 7*t^6 + t^4)*q^20 + (5*t^8 + 6*t^6 + t^4)*q^19 + (3*t^8 + 6*t^6 + t^4)*q^18 + (t^8 + 5*t^6 + t^4)*q^17 + (5*t^6 + t^4)*q^16 + (4*t^6 + t^4)*q^15 + (4*t^6 + t^4)*q^14 + (3*t^6 + t^4)*q^13 + (3*t^6 + t^4)*q^12 + q^9*t^4 + (2*t^6 + t^4)*q^11 + q^8*t^4 + (t^6 + t^4)*q^10 + q^7*t^4 + q^6*t^4 + q^5*t^4 + q^2*t^2

In [33]:
series_t_q(list_of_list_of_partition_ends(w, [4, 1, 1], True))

(12*t^8 + 8*t^6 + t^4)*q^27 + (10*t^8 + 8*t^6 + t^4)*q^26 + (8*t^8 + 7*t^6 + t^4)*q^25 + (6*t^8 + 7*t^6 + t^4)*q^24 + (5*t^8 + 6*t^6 + t^4)*q^23 + (3*t^8 + 6*t^6 + t^4)*q^22 + (2*t^8 + 5*t^6 + t^4)*q^21 + (t^8 + 5*t^6 + t^4)*q^20 + (4*t^6 + t^4)*q^19 + (4*t^6 + t^4)*q^18 + (3*t^6 + t^4)*q^17 + (3*t^6 + t^4)*q^16 + (2*t^6 + t^4)*q^15 + (2*t^6 + t^4)*q^14 + q^11*t^4 + (t^6 + t^4)*q^13 + q^10*t^4 + (t^6 + t^4)*q^12 + q^9*t^4 + q^8*t^4 + q^7*t^4 + q^2*t^2

In [34]:
series_t_q(list_of_list_of_partition_ends(w, [5, 4, 1, 1], True))

(5*t^8 + t^6)*q^27 + (5*t^8 + t^6)*q^26 + (4*t^8 + t^6)*q^25 + (4*t^8 + t^6)*q^24 + (3*t^8 + t^6)*q^23 + q^18*t^6 + (3*t^8 + t^6)*q^22 + q^17*t^6 + (2*t^8 + t^6)*q^21 + q^16*t^6 + (2*t^8 + t^6)*q^20 + q^15*t^6 + (t^8 + t^6)*q^19 + q^14*t^6 + q^13*t^6 + q^12*t^6 + q^6*t^4

In [35]:
series_t_q(list_of_list_of_partition_ends(w, [5, 4, 1, 1], False))

q^26*t^8 + q^25*t^8 + q^24*t^8 + q^23*t^8 + q^22*t^8 + q^21*t^8 + (t^10 + t^8)*q^27 + q^20*t^8 + q^19*t^8 + q^18*t^8 + q^11*t^6

In [36]:
series_t_q(list_of_list_of_partition_ends(w, [5, 3, 1, 1], True))

(t^10 + 6*t^8 + t^6)*q^27 + (5*t^8 + t^6)*q^26 + (5*t^8 + t^6)*q^25 + (4*t^8 + t^6)*q^24 + (4*t^8 + t^6)*q^23 + (3*t^8 + t^6)*q^22 + q^17*t^6 + (3*t^8 + t^6)*q^21 + q^16*t^6 + (2*t^8 + t^6)*q^20 + q^15*t^6 + (2*t^8 + t^6)*q^19 + q^14*t^6 + (t^8 + t^6)*q^18 + q^13*t^6 + q^12*t^6 + q^11*t^6 + q^5*t^4

In [37]:
series_t_q(list_of_list_of_partition_ends(w, [7, 5, 3, 1, 1], True))

q^25*t^8 + q^24*t^8 + q^23*t^8 + q^22*t^8 + q^21*t^8 + (t^10 + t^8)*q^27 + q^20*t^8 + (t^10 + t^8)*q^26 + q^19*t^8 + q^18*t^8 + q^10*t^6

In [38]:
series_t_q(list_of_list_of_partition_ends(w, [7, 5, 3, 1, 1], False))

q^27*t^10 + q^26*t^10 + q^17*t^8

In [39]:
series_t_q(list_of_list_of_partition_ends(w, [2, 1, 1, 1], True))

(14*t^9 + 9*t^7 + t^5)*q^27 + (11*t^9 + 8*t^7 + t^5)*q^26 + (9*t^9 + 8*t^7 + t^5)*q^25 + (7*t^9 + 7*t^7 + t^5)*q^24 + (5*t^9 + 7*t^7 + t^5)*q^23 + (3*t^9 + 6*t^7 + t^5)*q^22 + (2*t^9 + 6*t^7 + t^5)*q^21 + (5*t^7 + t^5)*q^20 + (5*t^7 + t^5)*q^19 + (4*t^7 + t^5)*q^18 + (4*t^7 + t^5)*q^17 + (3*t^7 + t^5)*q^16 + q^12*t^5 + (3*t^7 + t^5)*q^15 + q^11*t^5 + (2*t^7 + t^5)*q^14 + q^10*t^5 + (t^7 + t^5)*q^13 + q^9*t^5 + q^8*t^5 + q^7*t^5 + q^3*t^3

In [40]:
series_t_q(list_of_list_of_partition_ends(w, [4, 1, 1, 1], True))

(9*t^9 + 8*t^7 + t^5)*q^27 + (7*t^9 + 7*t^7 + t^5)*q^26 + (5*t^9 + 7*t^7 + t^5)*q^25 + (4*t^9 + 6*t^7 + t^5)*q^24 + (2*t^9 + 6*t^7 + t^5)*q^23 + (t^9 + 5*t^7 + t^5)*q^22 + (5*t^7 + t^5)*q^21 + (4*t^7 + t^5)*q^20 + (4*t^7 + t^5)*q^19 + (3*t^7 + t^5)*q^18 + (3*t^7 + t^5)*q^17 + q^13*t^5 + (2*t^7 + t^5)*q^16 + q^12*t^5 + (2*t^7 + t^5)*q^15 + q^11*t^5 + (t^7 + t^5)*q^14 + q^10*t^5 + q^9*t^5 + q^8*t^5 + q^3*t^3

In [41]:
series_t_q(list_of_list_of_partition_ends(w, [4, 1, 1, 1], False))

(5*t^9 + t^7)*q^27 + (4*t^9 + t^7)*q^26 + q^20*t^7 + (4*t^9 + t^7)*q^25 + q^19*t^7 + (3*t^9 + t^7)*q^24 + q^18*t^7 + (3*t^9 + t^7)*q^23 + q^17*t^7 + (2*t^9 + t^7)*q^22 + q^16*t^7 + (2*t^9 + t^7)*q^21 + q^15*t^7 + q^14*t^7 + q^13*t^7 + q^7*t^5